In [1]:
from collections import namedtuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import gym
import numpy as np

In [2]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
        
    def forward(self, x):
        return self.net(x)

In [4]:
# helper classes to track episodes
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    """
    generate batch of episodes
    """
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    # need to include manual softmax to convert nn output to probability distro
    # since using nn.CrossEntropyLoss which reqs raw, unnormalized data
    sm = nn.Softmax(dim=1)
    
    # env loop
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        step = EpisodeStep(observation=obs, action=action)
        obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        
        episode_steps.append(step)
        
        # if done, save and set up next
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            obs = env.reset()
            
            if len(batch) == batch_size:
                yield batch
                batch = []

In [6]:
def filter_batch(batch, percentile):
    """
    remove low performing episodes from batch for upcoming training step
    """
    rewards = list(map(lambda ep: ep.reward, batch))
    # create bound of reward acceptability
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))
    
    train_obs, train_act = [], []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))
        
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [7]:
# train
env = gym.make('CartPole-v0')
env = gym.wrappers.Monitor(env, directory='monitor', force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment='-cartpole')

for i, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_bound, reward_mean = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    
    print('%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f' %
         (i, loss_v.item(), reward_mean, reward_bound))
    writer.add_scalar('loss', loss_v.item(), i)
    writer.add_scalar('reward_bound', reward_bound, i)
    writer.add_scalar('reward_mean', reward_mean, i)
    
    if reward_mean > 199:
        print('Solved!')
        break
        
writer.close()

0: loss=0.690, reward_mean=22.9, rw_bound=32.0
1: loss=0.677, reward_mean=25.9, rw_bound=27.0
2: loss=0.666, reward_mean=33.6, rw_bound=36.0
3: loss=0.656, reward_mean=44.2, rw_bound=46.5
4: loss=0.644, reward_mean=42.6, rw_bound=42.0
5: loss=0.640, reward_mean=41.4, rw_bound=43.5
6: loss=0.630, reward_mean=35.3, rw_bound=32.5
7: loss=0.623, reward_mean=44.2, rw_bound=49.5
8: loss=0.604, reward_mean=47.6, rw_bound=53.0
9: loss=0.586, reward_mean=52.1, rw_bound=57.5
10: loss=0.585, reward_mean=63.0, rw_bound=66.0
11: loss=0.582, reward_mean=55.9, rw_bound=60.5
12: loss=0.563, reward_mean=49.5, rw_bound=58.5
13: loss=0.562, reward_mean=62.0, rw_bound=75.0
14: loss=0.550, reward_mean=61.6, rw_bound=73.0
15: loss=0.543, reward_mean=52.6, rw_bound=61.5
16: loss=0.547, reward_mean=54.4, rw_bound=58.0
17: loss=0.523, reward_mean=59.9, rw_bound=64.0
18: loss=0.528, reward_mean=68.9, rw_bound=70.0
19: loss=0.499, reward_mean=88.0, rw_bound=103.0
20: loss=0.506, reward_mean=91.0, rw_bound=109.0
